In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [9]:
file_name = os.path.join("data", "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")

# Read dataset
df = pd.read_csv(file_name, skipinitialspace=True, error_bad_lines=False)

C:\Users\olgas\AppData\Local\Temp\ipykernel_15264\4191304755.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_name, skipinitialspace=True, error_bad_lines=False)


In [10]:
# Rename incorrectly encoded labels
def _rename_class_label(df: pd.DataFrame):
    labels = {"Web Attack � Brute Force": "Web Attack-Brute Force",
              "Web Attack � XSS": "Web Attack-XSS",
              "Web Attack � Sql Injection": "Web Attack-Sql Injection"}

    for old_label, new_label in labels.items():
        df.Label.replace(old_label, new_label, inplace=True)

_rename_class_label(df)
df.to_csv(file_name, index=False)

In [11]:
# Set aside part of Port Scan dataset to create validation subset
def _create_validation_set(df_ps):
    df_ps = pd.read_csv("data/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")

    y = df_ps[' Label']
    x = df_ps.drop(columns=[' Label'])

    x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.15,
                                                        random_state=np.random.randint(10))

    del x, y

    train = pd.concat([x_train, y_train], axis=1, sort=False)
    test = pd.concat([x_test, y_test], axis=1, sort=False)

    train.to_csv('data/Friday-WorkingHours-Afternoon-PortScan-train.pcap_ISCX.csv', index=False, mode='a')
    test.to_csv('data/Friday-WorkingHours-Afternoon-PortScan-validation.pcap_ISCX.csv', index=False, mode='a')

df_ps = pd.read_csv("data/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
_create_validation_set(df_ps)

In [12]:
DIR_PATH = "data"

FILE_NAMES = ["Monday-WorkingHours.pcap_ISCX.csv",
              "Tuesday-WorkingHours.pcap_ISCX.csv",
              "Wednesday-workingHours.pcap_ISCX.csv",
              "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
              "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
              "Friday-WorkingHours-Morning.pcap_ISCX.csv",
              "Friday-WorkingHours-Afternoon-PortScan-train.pcap_ISCX.csv",
              "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"]

In [13]:
# Compile all datasets
df = [pd.read_csv(os.path.join(DIR_PATH, f), skipinitialspace=True) for f in FILE_NAMES]
df = pd.concat(df, ignore_index=True)
df.Label.value_counts()

BENIGN                      2214224
DoS Hulk                     231073
PortScan                     135090
DDoS                         128027
DoS GoldenEye                 10293
FTP-Patator                    7938
SSH-Patator                    5897
DoS slowloris                  5796
DoS Slowhttptest               5499
Bot                            1966
Web Attack-Brute Force         1507
Web Attack-XSS                  652
Infiltration                     36
Web Attack-Sql Injection         21
Heartbleed                       11
Name: Label, dtype: int64

In [14]:
df.to_csv(os.path.join(DIR_PATH, "MachineLearningCVE.csv"), index=False)